# Keyword Extraction

## Install Mecab

In [1]:
#!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
#!sudo apt-get install python-dev; pip install konlpy     # Python 2.x
#!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
#!sudo apt-get install curl
#!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# TF-IDF 활용 핵심키워드 추출

## sklearn 

In [2]:
import requests
from bs4 import BeautifulSoup
import re


def get_news_by_url(url):
    headers = {"user-agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    content = soup.select_one(
        "#articleBodyContents").get_text().replace("\n", "")
    content = content.replace(
        "// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")

    start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start()
    content = content[:start_pos-1]
    return content


docs = []
docs.append(get_news_by_url(
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108'))
docs.append(get_news_by_url(
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790'))
docs.append(get_news_by_url(
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362'))
docs.append(get_news_by_url(
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191'))
docs.append(get_news_by_url(
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728'))
print(len(docs))

5


### 1) 전처리

In [3]:
# 전처리
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []


for doc in docs:
    token_list = []
    for token in mecab.pos(doc):
        if token[1] in ['NNG', 'NNP', 'VV']:
            token_list.append(token[0])
    preprocessed_docs.append(" ".join(token_list))


preprocessed_docs[0][:100]

'과기 정통부 유영민 장관 참석 기념행사 투입 여종 데이터 구축 민간 클라우드 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 산업 창출 기존 산업 변'

### 2) TF-IDF 계산

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer(max_df=0.85, max_features=10000)
word_count = count_vect.fit_transform(preprocessed_docs)
print((count_vect.get_feature_names()[:10]))

['가공', '가능', '가입자', '가족', '가중치', '가치', '각종', '감소', '감염', '강국']


In [8]:
print(type(count_vect.fit(preprocessed_docs)))
print(type(count_vect.fit_transform(preprocessed_docs)))

<class 'sklearn.feature_extraction.text.CountVectorizer'>
<class 'scipy.sparse.csr.csr_matrix'>


In [9]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count)

TfidfTransformer()

### 3) 핵심키워드 추출

In [10]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [12]:
doc = preprocessed_docs[0]  # 핵심키워드 추출할 문서 조회

feature_names = count_vect.get_feature_names()
tfidf_vect = tfidf_transformer.transform(count_vect.transform([doc]))
sorted_keywords = sort_keywords(tfidf_vect.tocoo())


# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)

print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.25172034622187184)
('센터', 0.22285191065929333)
('계획', 0.21576029676160446)
('활용', 0.18233338144851272)
('정통부', 0.17980024730133706)


In [18]:
#tfidf_vect.tocoo().toarray()
#tfidf_vect.tocoo().col
#tfidf_vect.tocoo().data


---


## gensim

### 1) 전처리

In [19]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs:
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(
        ' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [20]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls)  # 인덱스(key) - 단어(valuue) 인 딕셔너리 생성
# 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
corpus = [dct.doc2bow(doc) for doc in document_ls]
tfidf = TfidfModel(corpus)  # TF-IDF 산출

/Users/dhkim/miniforge3/envs/nlp/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### 3) 핵심키워드 추출

In [21]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)


def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [22]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc])  # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.2495222182663338)
('폼', 0.2495222182663338)
('계획', 0.21387618708542896)
('정통부', 0.17823015590452412)
('위한', 0.17823015590452412)


In [23]:
#tfidf[doc]



---



# Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 행렬 활용 


In [24]:
tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
nodes = ['바나나', '사과', '파인애플', '딸기']
vocab = nodes

# [vocab2idx[token] for token in vocab] 

vocab2idx = {vocab[i]:i for i in range(0, len(vocab))} #vocab을 인덱스로 변환
idx2vocab = {i:vocab[i] for i in range(0, len(vocab))} #인덱스를 vocab으로 변환
vocab2idx

{'바나나': 0, '사과': 1, '파인애플': 2, '딸기': 3}

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [25]:
import numpy as np
import math
vocab_len = len(vocab)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len),dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 2
covered_cooccurence = []

tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']

for window_start in range(0, (len(tokens) - window_size + 1)):
    window = tokens[window_start : window_start + window_size]
    
    for i in range(window_size):
        for j in range(i + 1, window_size):
            if(window[i] in vocab and window[j] in vocab):
                index_i = i + window_start
                index_j = j + window_start
                
                if (index_i, index_j) not in covered_cooccurence:
                    weighted_edge[vocab2idx[window[i]]][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]][vocab2idx[window[i]]] = 1
                    covered_cooccurence.append((index_i, index_j))


for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    print(f"{i} : {row_sum}")
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

print(weighted_edge)

0 : 2.0
1 : 2.0
2 : 1.0
3 : 3.0
[[0.         0.5        0.         0.5       ]
 [0.5        0.         0.         0.5       ]
 [0.         0.         0.         1.        ]
 [0.33333334 0.33333334 0.33333334 0.        ]]


### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

In [27]:
MAX_ITERATIONS = 50
d = 0.85
threshold = 0.0001  # convergence threshold


for iter in range(MAX_ITERATIONS):
    # threshold 때문에 만듬
    prev_score = np.copy(score)

    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            summation += weighted_edge[j][i] * score[j]

        score[i] = (1-d) + d*summation
        print("***** score *****")
        print(score)
    # np.fabs flaot 절대값(abs)
    if np.sum(np.fabs(prev_score - score)) <= threshold:
        break
        
print("***** final *****")
print(score)
    

***** score *****
[0.98370874 0.9837155  0.56564105 1.4669281 ]
***** score *****
[0.98370874 0.9837059  0.56564105 1.4669281 ]
***** score *****
[0.98370874 0.9837059  0.56562966 1.4669281 ]
***** score *****
[0.98370874 0.9837059  0.56562966 1.4669365 ]
***** final *****
[0.98370874 0.9837059  0.56562966 1.4669365 ]


### 5) 핵심 단어 추출

In [28]:
# np.argsort() <-- sort array and return index
sorted_index = np.flip(np.argsort(score), 0)

n = 4

print("\n=== 핵심키워드 ===")
for i in range(0, n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
딸기 : 1.4669365
바나나 : 0.98370874
사과 : 0.9837059
파인애플 : 0.56562966


In [29]:
# sorted_index[0]

In [31]:
import requests
from bs4 import BeautifulSoup
import re


def get_news_by_url(url):
    headers = {"user-agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, "html.parser")
    content = soup.select_one(
        "#articleBodyContents").get_text().replace("\n", "")
    content = content.replace(
        "// flash 오류를 우회하기 위한 함수 추가function _flash_removeCallback() {}", "")

    start_pos = re.search(r"\w+@\w+\.\w+(.\w+)?", content).start()
    content = content[:start_pos-1]
    return content


doc = get_news_by_url(
    'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc = re.sub("[^가-힣 \d]", " ", doc)
#doc

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [32]:
from konlpy.tag import Mecab
mecab = Mecab()

tokens = [ token for token in mecab.pos(doc) ]
nodes = [t[0] for t in tokens]
vocab = [t[0] for t in tokens if t[1] in ['NNG', 'NNP'] and len(t[0]) > 1]

print(nodes[:10])
print(vocab[:10])

['과기', '정통부', '22', '일', '유영민', '장관', '등', '참석', '해', '기념행사']
['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '데이터', '구축']


### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [33]:
vocab = list(set(vocab))

vocab2idx = {vocab[i]: i for i in range(len(vocab))}
idx2vocab = {i: vocab[i] for i in range(len(vocab))}

In [34]:
import numpy as np
import math

vocab_len = len(vocab2idx)

# 토큰별로 그래프 edge를 Matrix 형태로 생성
weighted_edge = np.zeros((vocab_len, vocab_len), dtype=np.float32)

# 각 토큰 노드별로 스코어 1로 초기화
score = np.ones((vocab_len), dtype=np.float32)

# coocurrence를 판단하기 위한 window 사이즈 설정
window_size = 3
covered_coocurrences = []

for window_start in range(len(nodes) - window_size + 1):
    window = nodes[window_start:window_start+window_size]
    for i in range(window_size):
        for j in range(i+1, window_size):
            if window[i] in vocab and window[j] in vocab:
                index_i = window_start + i
                index_j = window_start + j

                if (index_i, index_j) not in covered_coocurrences:
                    weighted_edge[vocab2idx[window[i]]
                                  ][vocab2idx[window[j]]] = 1
                    weighted_edge[vocab2idx[window[j]]
                                  ][vocab2idx[window[i]]] = 1
                    covered_coocurrences.append((index_i, index_j))

for i in range(vocab_len):
    row_sum = weighted_edge[i].sum()
    weighted_edge[i] = weighted_edge[i]/row_sum if row_sum > 0 else 0

MAX_ITERATIONS = 50
d = 0.85
threshold = 0.0001  # convergence threshold

for iter in range(MAX_ITERATIONS):
    prev_score = np.copy(score)

    for i in range(vocab_len):
        summation = 0
        for j in range(vocab_len):
            if weighted_edge[j][i] != 0:
                summation += weighted_edge[j][i] * prev_score[j]

        score[i] = (1 - d) * d*summation

    if np.sum(np.fabs(prev_score - score)) <= threshold:
        break


sorted_index = np.flip(np.argsort(score), 0)

n = 5


print("\n=== 핵심키워드 ===")
for i in range(0, n):
    print(str(idx2vocab[sorted_index[i]])+" : " + str(score[sorted_index[i]]))


=== 핵심키워드 ===
데이터 : 7.405189e-07
센터 : 2.49831e-07
한국 : 2.0378042e-07
활용 : 1.9834519e-07
대한 : 1.9389464e-07


---

## 그래프 활용 

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

### 2) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 없음을 의미
*   모든 노드는 1로 초기화

In [35]:
import numpy as np
import math
import networkx as nx

# 윈도내 동시 등장한 토큰으로 그래프를 생성


def connect(vocab, nodes):
    window_size = 3
    
    edges = []
    for window_start in range(len(nodes)-window_size +1):
        window = nodes[window_start: window_start + window_size]
        for i in range(window_size):
            for j in range(i + 1,window_size):
                if window[i] in vocab and window[j] in vocab:
                    edges.append((window[i], window[j]))
    return edges

graph = nx.diamond_graph()
graph.clear()
graph.add_nodes_from(list(set(nodes)))  # node 등록
graph.add_edges_from(connect(nodes, tokens))  # edge 연결

### 3) 스코어 계산 및 핵심키워드 추출

In [36]:
scores = nx.pagerank(graph)  # pagerank 계산
rank = sorted(scores.items(), key=lambda x: x[1], reverse=True)  # score 역순 정렬
print("\n=== 핵심키워드 ===")
rank[:5]


=== 핵심키워드 ===


[('물론', 0.0027472527472527405),
 ('문', 0.0027472527472527405),
 ('출범식', 0.0027472527472527405),
 ('최신', 0.0027472527472527405),
 ('생태', 0.0027472527472527405)]